Hello everyone!

This is the notebook for our final project! I'm committing this file to the repo so that everyone can start from the same spot.

Few questions:

1) Video call meetups?

2) Regarding Git/GitHub, how comfortable are you all with checking out your own branches to work and then pushing the changes to the remote master branch?

I'd appreciate it if you could show me how in person. -ZG

In [3]:
import json
import billboard as bd
from sys import exit, argv
from pathlib import Path
from os import remove

In [4]:
def grab_data(current_chart):

    chart_data = {}

    # Loop through the chart populating the dict
    for ind in range(len(current_chart)):

        # Populate the list
        attrib = []
        attrib.append(current_chart[ind].title)
        attrib.append(current_chart[ind].artist)
        attrib.append(current_chart[ind].peakPos)
        attrib.append(current_chart[ind].lastPos)
        attrib.append(current_chart[ind].weeks)

        # Append the list into the dict for the rank as the key val
        chart_data[current_chart[ind].rank] = attrib

    return chart_data

In [5]:
def create_json(list_to_use, file_to_use):

    with open(file_to_use, 'a') as outputfile:

        json.dump(list_to_use, outputfile)

In [6]:
def main():


    # Check command line argument number for correctness. Should only be 1
    if len(argv) != 2:

        print('Expected a file name as the only argument.')
        exit(1)


    # Parse the command line argument as the name of the file to save data to
    file_to_use = argv[1]

    # Make sure string passed in ends with .json
    if '.json' not in file_to_use:

        file_to_use += '.json'

    # Create a path to the file so that we can remove incomplete data if
    # interrupted
    next_filename = './' + file_to_use
    my_file = Path(next_filename)

    # Grab current chart
    current_chart = bd.ChartData('hot-100', date='2018-02-24')

    # Create a empty list of dicts of charts
    data = []

    try:

        # Set up our loop to download all the data at once
        while current_chart.previousDate:

            print('Grabbing chart data %s as a dict...\n' % current_chart.date)

            # Grab the data and save it into a dict
            next_chart = grab_data(current_chart)

            # append that dict to the list of dicts
            data.append(next_chart)

            # Update current chart to get next set of data
            current_chart = bd.ChartData('hot-100', current_chart.previousDate)

        # Dump that new list of dicts into the file given
        print('Appending to .json file...\n')
        create_json(data, file_to_use)

        print('Complete!\n')

    except (EOFError, KeyboardInterrupt):

        print('\nInterrupted. Removing incomplete data file and exiting...\n')

        # Delete the file before exiting
        if my_file.exists():

            remove(my_file)

        exit(1)


# Standard boilerplate to run main
if __name__ == '__main__':
    main()

Expected a file name as the only argument.


SystemExit: 1

/Users/zachariahgutierrez/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import pandas as pd

In [3]:
testdf = pd.read_json('project_data.json')

In [4]:
testdf

,1,10,100,11,12,13,14,15,16,17,...,90,91,92,93,94,95,96,97,98,99
0,"[God's Plan, Drake, 1, 1, 4]","[Stir Fry, Migos, 8, 8, 8]","[Rewrite The Stars, Zac Efron & Zendaya, 70, 8...","[Pray For Me, The Weeknd & Kendrick Lamar, 7, ...","[Let You Down, NF, 12, 12, 18]","[Love., Kendrick Lamar Featuring Zacari, 11, 1...","[Him & I, G-Eazy & Halsey, 14, 15, 11]","[Thunder, Imagine Dragons, 4, 11, 42]","[Bad At Love, Halsey, 5, 10, 24]","[The Middle, Zedd, Maren Morris & Grey, 17, 22...",...,"[Filthy, Justin Timberlake, 9, 34, 6]","[Black Panther, Kendrick Lamar, 91, 0, 1]","[At The Club, Jacquees X Dej Loaf, 92, 92, 2]","[Codeine Dreaming, Kodak Black Featuring Lil W...","[The Long Way, Brett Eldredge, 93, 93, 2]","[Rock, Plies, 95, 99, 3]","[I Like Me Better, Lauv, 96, 0, 1]","[When We, Tank, 86, 0, 7]","[Criminal, Natti Natasha x Ozuna, 95, 95, 3]","[IDGAF, Dua Lipa, 88, 88, 5]"
1,"[God's Plan, Drake, 1, 1, 3]","[Bad At Love, Halsey, 5, 6, 23]","[Supplies, Justin Timberlake, 71, 0, 2]","[Thunder, Imagine Dragons, 4, 10, 41]","[Let You Down, NF, 12, 14, 17]","[Love., Kendrick Lamar Featuring Zacari, 11, 1...","[MotorSport, Migos, Nicki Minaj & Cardi B, 6, ...","[Him & I, G-Eazy & Halsey, 15, 16, 10]","[No Limit, G-Eazy Featuring A$AP Rocky & Cardi...","[Say Something, Justin Timberlake Featuring Ch...",...,"[Hardaway, Derez De'Shon, 90, 0, 1]","[Mayores, Becky G Featuring Bad Bunny, 74, 95,...","[At The Club, Jacquees X Dej Loaf, 92, 0, 1]","[The Long Way, Brett Eldredge, 93, 0, 1]","[Singles You Up, Jordan Davis, 94, 0, 1]","[Criminal, Natti Natasha x Ozuna, 95, 0, 2]","[Corazon, Maluma X Nego do Borel, 87, 100, 6]","[Sick Boy, The Chainsmokers, 65, 90, 3]","[A Girl Like You, Easton Corbin, 98, 0, 1]","[Rock, Plies, 99, 0, 2]"
2,"[God's Plan, Drake, 1, 1, 2]","[Thunder, Imagine Dragons, 4, 9, 40]","[Corazon, Maluma X Nego do Borel, 87, 93, 5]","[Meant To Be, Bebe Rexha & Florida Georgia Lin...","[Stir Fry, Migos, 12, 52, 6]","[No Limit, G-Eazy Featuring A$AP Rocky & Cardi...","[Let You Down, NF, 14, 14, 16]","[Love., Kendrick Lamar Featuring Zacari, 11, 1...","[Him & I, G-Eazy & Halsey, 15, 16, 9]","[I Fall Apart, Post Malone, 16, 19, 19]",...,"[Sick Boy, The Chainsmokers, 65, 65, 2]","[Rubbin Off The Paint, YBN Nahmir, 46, 69, 13]","[El Farsante, Ozuna & Romeo Santos, 92, 0, 1]","[IDGAF, Dua Lipa, 92, 92, 3]","[Most People Are Good, Luke Bryan, 94, 0, 1]","[Mayores, Becky G Featuring Bad Bunny, 74, 94,...","[CC, Migos Featuring Gucci Mane, 96, 0, 1]","[Best Friend, Sofi Tukker Featuring NERVO, The...","[MIC Drop, BTS Featuring Desiigner, 28, 84, 10]","[King's Dead, Jay Rock, Kendrick Lamar, Future..."
3,"[God's Plan, Drake, 1, 0, 1]","[No Limit, G-Eazy Featuring A$AP Rocky & Cardi...","[Female, Keith Urban, 69, 100, 5]","[MotorSport, Migos, Nicki Minaj & Cardi B, 6, ...","[Meant To Be, Bebe Rexha & Florida Georgia Lin...","[Too Good At Goodbyes, Sam Smith, 4, 9, 20]","[Let You Down, NF, 14, 14, 15]","[Love., Kendrick Lamar Featuring Zacari, 11, 1...","[Him & I, G-Eazy & Halsey, 15, 15, 8]","[Gucci Gang, Lil Pump, 3, 11, 20]",...,"[Ice Tray, Quavo & Lil Yachty, 74, 86, 6]","[Beautiful Trauma, P!nk, 91, 99, 5]","[IDGAF, Dua Lipa, 92, 92, 2]","[Corazon, Maluma X Nego do Borel, 87, 87, 4]","[Mayores, Becky G Featuring Bad Bunny, 74, 93,...","[The Greatest Show, Hugh Jackman, Keala Settle...","[Never Enough, Loren Allred, 88, 97, 4]","[My Dawg, Lil Baby, 71, 90, 7]","[I'll Name The Dogs, Blake Shelton, 56, 82, 20]","[Rock, Plies, 99, 0, 1]"
4,"[Havana, Camila Cabello Featuring Young Thug, ...","[MotorSport, Migos, Nicki Minaj & Cardi B, 6, ...","[Female, Keith Urban, 69, 96, 4]","[Gucci Gang, Lil Pump, 3, 12, 19]","[Love., Kendrick Lamar Featuring Zacari, 11, 1...","[Gummo, 6ix9ine, 12, 19, 10]","[Let You Down, NF, 14, 15, 14]","[Him & I, G-Eazy & Halsey, 15, 16, 7]","[Bartier Cardi, Cardi B Featuring 21 Savage, 1...","[Meant To Be, Bebe Rexha & Florida Georgia Lin...",...,"[My Dawg, Lil Baby, 71, 87, 6]","[King's Dead, Jay Rock, Kendri